In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import glob
import pathlib
from palettable.wesanderson import Moonrise5_6

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9r68bdbv because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
files = glob.glob("data/*gpkg")
cases = pd.read_csv("data/case_studies.csv")
cases = cases.set_index("case")

In [3]:
%%capture --no-stdout

for f in files:
    tessellation = gpd.read_file(f, layer="tessellation")
    buildings = gpd.read_file(f, layer="buildings")
    edges = gpd.read_file(f, layer="edges")
    nodes = gpd.read_file(f, layer="nodes")
    case = pathlib.Path(f).stem
    coords = cases.origin.loc[case]
    buffer = gpd.GeoSeries([Point(tuple(map(float, coords[1:-1].split(', '))))], crs=4326).to_crs(tessellation.crs).buffer(400)
    ax = gpd.clip(buildings, buffer).plot(zorder=1, color=Moonrise5_6.hex_colors[3], figsize=(12, 12))
    buildings.plot(ax=ax, zorder=0, color=Moonrise5_6.hex_colors[3], alpha=.2)
    buffer.plot(ax=ax, facecolor="none", edgecolor=Moonrise5_6.hex_colors[4], linewidth=2)
    ax.set_axis_off()
    b = buffer.total_bounds
    ax.set_xlim(b[0]-20, b[2]+20)
    ax.set_ylim(b[1]-20, b[3]+20)
    plt.savefig(f"figures/{case}.png", bbox_inches="tight")
    plt.close("all")